# Securing Your Randomness

If you don't want your agent to be attacked with any RNG search method, you can add some simple code that is cryptographically secure and won't be able to be cracked. This can be done using the `secrets` module and the `SystemRandom()` class. `SystemRandom` uses the same interface as the normal `random` library, so the functions will work the same (e.g. `randint`, `choice`, etc.)

In [ ]:
%%writefile safe_random.py

# Importing the secrets module
import secrets

# Initializing the random lib
# Will have the same functions as normal random
random = secrets.SystemRandom()

# Defining Kaggle format agent
def safe_random_agent(obs, config):
    
    # Returning secure random action in the environment range
    return random.randrange(config.signs)

# Random Agent Solver

This notebook attempts a standard method for solving pseudo-random number generators. Since all computer random number generators are not completely random, they can be predicted. This notebook goes over some ways to try and predict random agent's moves based by cracking the Mersenne Twister algorithm.

Inspired by: https://www.kaggle.com/c/rock-paper-scissors/discussion/195923

# Method I: Random Number Cracker

Using a random number cracker to try and predict the next opponent move and exploit it.

# Cracker Library

The random number cracker is a copy of [this GitHub repo](https://github.com/tna0y/Python-random-module-cracker) which can be used to solve for the Mersenne Twister random number generator algorithm which is used in `random.randint`, `random.choice`, `np.random.randint`, `np.random.choice`, etc.

In [ ]:
class RandomCracker:

	def __init__(self):
		self.counter = 0
		self.mt = []
		self.state = False

	def submit(self, num):
		if self.state:
			raise ValueError("Already got enough bits")

		bits = self._to_bitarray(num)

		assert (all([x == 0 or x == 1 for x in bits]))
		self.counter += 1
		self.mt.append(self._harden_inverse(bits))
		if self.counter == 624:
			self._regen()
			self.state = True

	def _predict_32(self):
		if not self.state:
			raise ValueError("Didn't recieve enough bits to predict")

		if self.counter >= 624:
			self._regen()
		self.counter += 1

		return self._harden(self.mt[self.counter - 1])

	def predict_getrandbits(self, k):
		if not self.state:
			raise ValueError("Didn't recieve enough bits to predict")

		if k == 0:
			return 0
		words = (k - 1) // 32 + 1
		res = []
		for i in range(words):
			r = self._predict_32()
			if k < 32:
				r = [0] * (32 - k) + r[:k]
			res = r + res
			k -= 32
		return self._to_int(res)

	def predict_randbelow(self, n):
		k = n.bit_length()
		r = self.predict_getrandbits(k)
		while r >= n:
			r = self.predict_getrandbits(k)
		return r

	def predict_randrange(self, start, stop=None, step=1, _int=int):
		# Adopted messy code from random.py module
		# In fact only changed _randbelow() method calls to predict_randbelow()
		istart = _int(start)
		if istart != start:
			raise ValueError("non-integer arg 1 for randrange()")
		if stop is None:
			if istart > 0:
				return self.predict_randbelow(istart)
			raise ValueError("empty range for randrange()")

		# stop argument supplied.
		istop = _int(stop)
		if istop != stop:
			raise ValueError("non-integer stop for randrange()")
		width = istop - istart
		if step == 1 and width > 0:
			return istart + self.predict_randbelow(width)
		if step == 1:
			raise ValueError("empty range for randrange() (%d,%d, %d)" % (istart, istop, width))

		# Non-unit step argument supplied.
		istep = _int(step)
		if istep != step:
			raise ValueError("non-integer step for randrange()")
		if istep > 0:
			n = (width + istep - 1) // istep
		elif istep < 0:
			n = (width + istep + 1) // istep
		else:
			raise ValueError("zero step for randrange()")

		if n <= 0:
			raise ValueError("empty range for randrange()")

		return istart + istep * self.predict_randbelow(n)

	def predict_randint(self, a, b):
		return self.predict_randrange(a, b + 1)

	def predict_choice(self, seq):
		try:
			i = self.predict_randbelow(len(seq))
		except ValueError:
			raise IndexError('Cannot choose from an empty sequence')
		return seq[i]

	def _to_bitarray(self, num):
		k = [int(x) for x in bin(num)[2:]]
		return [0] * (32 - len(k)) + k

	def _to_int(self, bits):
		return int("".join(str(i) for i in bits), 2)

	def _or_nums(self, a, b):
		if len(a) < 32:
			a = [0] * (32 - len(a)) + a
		if len(b) < 32:
			b = [0] * (32 - len(b)) + b

		return [x[0] | x[1] for x in zip(a, b)]

	def _xor_nums(self, a, b):
		if len(a) < 32:
			a = [0] * (32 - len(a)) + a
		if len(b) < 32:
			b = [0] * (32 - len(b)) + b

		return [x[0] ^ x[1] for x in zip(a, b)]

	def _and_nums(self, a, b):
		if len(a) < 32:
			a = [0] * (32 - len(a)) + a
		if len(b) < 32:
			b = [0] * (32 - len(b)) + b

		return [x[0] & x[1] for x in zip(a, b)]

	def _decode_harden_midop(self, enc, and_arr, shift):

		NEW = 0
		XOR = 1
		OK = 2
		work = []
		for i in range(32):
			work.append((NEW, enc[i]))
		changed = True
		while changed:
			changed = False
			for i in range(32):
				status = work[i][0]
				data = work[i][1]
				if i >= 32 - shift and status == NEW:
					work[i] = (OK, data)
					changed = True
				elif i < 32 - shift and status == NEW:
					if and_arr[i] == 0:
						work[i] = (OK, data)
						changed = True
					else:
						work[i] = (XOR, data)
						changed = True
				elif status == XOR:
					i_other = i + shift
					if work[i_other][0] == OK:
						work[i] = (OK, data ^ work[i_other][1])
						changed = True

		return [x[1] for x in work]

	def _harden(self, bits):
		bits = self._xor_nums(bits, bits[:-11])
		bits = self._xor_nums(bits, self._and_nums(bits[7:] + [0] * 7, self._to_bitarray(0x9d2c5680)))
		bits = self._xor_nums(bits, self._and_nums(bits[15:] + [0] * 15, self._to_bitarray(0xefc60000)))
		bits = self._xor_nums(bits, bits[:-18])
		return bits

	def _harden_inverse(self, bits):
		# inverse for: bits = _xor_nums(bits, bits[:-11])
		bits = self._xor_nums(bits, bits[:-18])
		# inverse for: bits = _xor_nums(bits, _and_nums(bits[15:] + [0] * 15 , _to_bitarray(0xefc60000)))
		bits = self._decode_harden_midop(bits, self._to_bitarray(0xefc60000), 15)
		# inverse for: bits = _xor_nums(bits, _and_nums(bits[7:] + [0] * 7 , _to_bitarray(0x9d2c5680)))
		bits = self._decode_harden_midop(bits, self._to_bitarray(0x9d2c5680), 7)
		# inverse for: bits = _xor_nums(bits, bits[:-11])
		bits = self._xor_nums(bits, [0] * 11 + bits[:11] + [0] * 10)
		bits = self._xor_nums(bits, bits[11:21])

		return bits

	def _regen(self):
		# C code translated from python sources
		N = 624
		M = 397
		MATRIX_A = 0x9908b0df
		LOWER_MASK = 0x7fffffff
		UPPER_MASK = 0x80000000
		mag01 = [self._to_bitarray(0), self._to_bitarray(MATRIX_A)]

		l_bits = self._to_bitarray(LOWER_MASK)
		u_bits = self._to_bitarray(UPPER_MASK)

		for kk in range(0, N - M):
			y = self._or_nums(self._and_nums(self.mt[kk], u_bits), self._and_nums(self.mt[kk + 1], l_bits))
			self.mt[kk] = self._xor_nums(self._xor_nums(self.mt[kk + M], y[:-1]), mag01[y[-1] & 1])

		for kk in range(N - M - 1, N - 1):
			y = self._or_nums(self._and_nums(self.mt[kk], u_bits), self._and_nums(self.mt[kk + 1], l_bits))
			self.mt[kk] = self._xor_nums(self._xor_nums(self.mt[kk + (M - N)], y[:-1]), mag01[y[-1] & 1])

		y = self._or_nums(self._and_nums(self.mt[N - 1], u_bits), self._and_nums(self.mt[0], l_bits))
		self.mt[N - 1] = self._xor_nums(self._xor_nums(self.mt[M - 1], y[:-1]), mag01[y[-1] & 1])

		self.counter = 0

print('Random Cracker Library Loaded')

def play(agent1, agent2, render = True):
    
	from kaggle_environments import make

	env = make('rps', debug = True)
	env.reset()

	env.run([agent1, agent2])
	if render: env.render(mode = 'ipython', width = 500, height = 400)

	rewards = env.toJSON()['rewards']
	print(f'Rewards {rewards}')

In [ ]:
import random

def random_agent(obs, config):
    ''' Pure random agent '''
    return random.randint(0, 2)

# Solver Agent

This agent plays randomly for the first 625 moves as the cracker gathers data based on the assumed random opponent. It then predicts the seed of the next move for the opponent and beats that move.

In [ ]:

cracker = RandomCracker()

def beat_random_agent(obs, config):
    
    ''' Predicting a random opponent move '''
    
    global cracker
    
    if obs.step < 625:
        if obs.step: cracker.submit(obs.lastOpponentAction)
        return 0
    
    predicted_move = cracker.predict_choice([0, 1, 2])
    return (predicted_move + 1) % 3

In [ ]:
play(beat_random_agent, random_agent, render = False)

# Why Did it Fail?

You can see that the results of the test aren't good. One would assume that the `beat_random_agent` would completely demolish the pure random bot by 400+ points. This is not the case, and you can see that the game is very close to just a random bot playing another random bot. This is because the cracker only works well if it gets a broad range of data. With the relatively small range of 0 to 2 integer values, the cracker can't accuratly predict the next opponent move because there isn't enough data.

As outlined in [this comment](https://www.kaggle.com/c/rock-paper-scissors/discussion/195923#1072874), the cracker isn't as accurate when simply playing the game, but it can become super accurate if you load the training data in before the game actually starts.

# Pre-Training

In this runthrough, the agent will be given the random data in advance, and then will play the game. This should drastically increase it's accuracy because it is given a little headstart in the opponents mind.

In [ ]:

import random

# Initializing the random cracker
cracker = RandomCracker()

def beat_random_agent(obs, config):
    
    ''' 
    Getting the state of the opponent and predicting their
    moves from this (this won't work in-game because the code is isolated)
    '''
    
    global cracker
    
    # Loading the cracker on the first move
    if obs.step == 0:
        for i in range(624):
            cracker.submit(random.randint(0, 4294967294))
            
    predicted_move = cracker.predict_randint(0, 2)
    return (predicted_move + 1) % 3

In [ ]:
play(beat_random_agent, random_agent)

# Method I Conclusion

As you can see, that test went a lot better, beating the random bot. This shows how it is possible to beat the random bot, but only with pre-training the cracker on a larger dataset. This is kind of cheating, as this method won't be available in the game, but this opens a discussion to try and find other ways of trying to beat the avalanche of unbeatable random bots in this competition.


# Method II: Seed Overwriter

Overwriting the seed of the opponent and exploiting the fact that you can control their moves

In [ ]:
%%writefile random_agent.py

'''
Writing this agent to an isolated file
to see if the global seed overwrite can
affect it in this environment
'''

import random

def agent(obs, config):
    ''' Redefining a random bot just in case you forgot :) '''
    return random.randint(0, 2)

In [ ]:
%%writefile seed_overwriter.py

''' Also isolating this to a file '''

# Imports
import numpy as np
import random

# Global Variables
matrix = np.zeros((3, 3), dtype = int)
last_action = None

def agent(obs, config):
    
    ''' Checking if the opponent is random, then overwriting their seed '''
    
    # Global Variables
    global last_action
    global matrix
    
    # Playing Uniform randomness at the start
    random.seed(np.random.random())
    last_action = random.randint(0, 2)
    
    # Setting a universal seed
    random.seed(1337)

    # Filling up the table based on moves
    if obs.step and last_action != None:
        matrix[last_action][obs.lastOpponentAction] += 1
        
        # After adequate data has been gathered
        if obs.step > 200:
            
            # Percentage data (if there is an outlier, then we know the seed worked)
            percentages = np.zeros((3, 3), dtype = float)
            for row in range(3):
                for col in range(3):
                    percentages[row][col] = matrix[row][col] / sum(matrix[row])
            
            # Loop through responses
            for row in range(3):
                
                # Higher distribution of this value
                if max(percentages[row]) > 0.7:
                    
                    # Simulating their next move based on this
                    next_move = random.randint(0, 2)
                    last_action = (next_move + 1) % 3
                    break
                    
    # Returning a move    
    return last_action

In [ ]:
play('/kaggle/working/seed_overwriter.py', '/kaggle/working/random_agent.py')

# Method II Conclusion

Wait, that actually worked? I'm surprised that we can actually overwrite the seed for random bots and then exploit them based on the fact that we can control their moves. In the submission environment, our code will be sandboxed in individual GVisor containers, so I doubt that the overwriting will work. 

I personally, will not be submitting this bot as I don't think that it is in the spirit of the game, and because it might be breaking some rules about interferring in the opponent's environment. I just thought that sharing this might be interesting to see the limits of how we can exploit pseudo-random number generators.

# Method III: Seed Searcher

Based on [this notebook](https://www.kaggle.com/jamesmcguigan/rock-paper-scissors-random-seed-search/) we can try and guess the opponent seed and use that. Here we need to collect some data on their moves, and using that we can find a seed that matches. This requires knowing the range of the seed, as well as the datatype (int or float). It is infeasable to look through every single floating point number at 20 decimal places for this small experiment, so for this we can just stick to an integer between 0 and 2000.

In [ ]:
%%writefile random_agent.py

import random

# Arbitrary value
random.seed(1337)

def random_agent(obs, config):
    ''' A random bot '''
    return random.randint(0, 2)

In [ ]:
%%writefile seed_searcher.py

# Importing important imports
import numpy as np
import random

# Global Variables
seeds = list(range(2000))
previous_moves = []

def seed_searcher(obs, config):
    
    ''' An agent that searches for the seed of the opponent '''
    
    # Global Variables
    global previous_moves
    global seeds
    
    # Saving the current state
    init_state = random.getstate()
    
    # Initializing a backup move
    next_move = int(np.random.randint(3))
    
    # If there still are multiple canditates
    if obs.step and len(seeds) > 1:
        
        # Saving previous moves
        previous_moves.append(obs.lastOpponentAction)
        
        # Checking each possible seed
        for i in range(len(seeds) - 1, -1, -1):
            
            # Running for previous moves
            random.seed(seeds[i])
            for _ in range(obs.step):
                move = random.randint(0, 2)
           
            # Testing their move order
            if move != previous_moves[-1]:
                seeds.pop(i)
                
    # Seed found: Get the next move
    elif len(seeds) == 1:
        random.seed(seeds[0])
        for _ in range(obs.step):
            move = random.randint(0, 2)
        next_move = random.randint(0, 2)
            
    # Reseting the state to not interfer with the opponent
    random.setstate(init_state)
    
    # Returning an action
    return (next_move + 1) % 3

In [ ]:
play('/kaggle/working/seed_searcher.py', '/kaggle/working/random_agent.py')

# Method III Conclusion

As you can see, this method also can consistantly work in beating the random number generator by narrowing down a list of canditates that could possibly be the seed. This will only work with a random number generator that has a seed of integer 0 to N. This can be adapted to include all floats between 0 and 1, but that would be computationally infeasable at the moment with the current algorithm. Major optimizations would be needed to get that seed, but this seems good enough for right now. I'll probably expand on this later on as this method seems to be the most promising method of beating random number generators.

# Final Conclusion

I made this notebook to see if we can exploit random number generators and try and get an edge over them in this game. So far, none of these ideas have proved consistanly successful and/or legal, but I'll keep searching. I just hope that this opens up new ways to try and exploit random number generators and take back the leaderboard!


Some things to think about:
* By default, the seed for the random library if not specified uses the current system time (`time.time()`). If you can estimate when the opponent sent their move, you can try and guess their seed.
* Some other algorithms have been made to try and solve the Mersenne Twister, and a resource can be found [here](https://jazzy.id.au/2010/09/22/cracking_random_number_generators_part_3.html).
* Random bots can easily bypass all this by resetting their seed to a different value each time, so most of this is probably overkill for something that is easily avoidable

Suggestions and feedback is welcome! Good Luck!